In [1]:
## Import relevant libraries
import sys
import glob

sys.path.append('../../py_files/')
import quadrop as qd

qd.set_plotting_style()

### Data pre-procesing

In [2]:
# # Example usage
# base_dir = "../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-160nM_80nM-RT/"
# qd.consolidate_images(base_dir)

In [11]:
# Example usage
data_path = "../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/"

calibration_curve_paths = sorted(glob.glob("../../../../Thomson Lab Dropbox/David Larios/activedrops/calibration-50ms/***ugml.tif"))


conditions_dict = {
    "D_160nM": "Pos0", 
    "D_80nM": "Pos1",
    "D_40nM": "Pos2",
    "D_20nM": "Pos3",
    "D_10nM": "Pos4",
    "D_5nM": "Pos5",
    "D_2p5nM": "Pos6",
    "D_1p25nM": "Pos7",
    "D_0p625nM": "Pos8",
    "D_0p312nM": "Pos9",
    "D_0p156nM": "Pos10",
    "negative": "Pos11",
}

# Organize PosX folders into condition folders
qd.organize_conditions(data_path, conditions_dict)

# Now run the existing functions to reorganize the tiffs and rename the folders
conditions, subconditions = qd.prepare_conditions(data_path)
time_interval_list = [8] * len(conditions)  # time intervals in seconds between frames for each condition

print("Conditions:", conditions)
print("Subconditions:", subconditions)



Conditions: ['D_0p156nM', 'D_0p312nM', 'D_0p625nM', 'D_1p25nM', 'D_2p5nM', 'D_5nM', 'D_10nM', 'D_20nM', 'D_40nM', 'D_80nM', 'D_160nM', 'negative']
Subconditions: ['Rep1']


In [4]:
qd.reorgTiffsToOriginal(data_path, conditions, subconditions)


Moved .tif files from ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/D_0p156nM/Rep1 to ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/D_0p156nM/Rep1/original
Moved .tif files from ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/D_0p312nM/Rep1 to ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/D_0p312nM/Rep1/original
Moved .tif files from ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/D_0p625nM/Rep1 to ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/D_0p625nM/Rep1/original
Moved .tif files from ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/D_1p25nM/Rep1 to ../../../../Thomson Lab Dropbox/David Larios/activedrop

### Generate movies

In [8]:
conditions = conditions[::-1]

In [9]:
# Call the function
qd.fluorescence_heatmap(
    data_path, 
    conditions, 
    subconditions, 
    channel='cy5', 
    time_interval_list=time_interval_list, 
    vmax=12, 
    skip_frames=8, 
    calibration_curve_paths=calibration_curve_paths, 
    show_scalebar=False,
    )

Processing D_0p156nM - Rep1: 100%|██████████| 338/338 [00:42<00:00,  7.98it/s]


In [10]:
# Example usage
qd.create_movies(
    data_path, 
    conditions, 
    subconditions, 
    channel='cy5', 
    frame_rate=24,
    skip_frames=1
    )


Creating video for negative - Rep1 with duration: 14.08 seconds.


Creating video for negative - Rep1: 100%|██████████| 338/338 [00:23<00:00, 14.11it/s]


Creating video for D_160nM - Rep1 with duration: 14.08 seconds.


Creating video for D_160nM - Rep1: 100%|██████████| 338/338 [00:25<00:00, 13.07it/s]


Creating video for D_80nM - Rep1 with duration: 14.08 seconds.


...

Creating video for D_0p625nM - Rep1: 100%|██████████| 338/338 [00:24<00:00, 13.64it/s]


Creating video for D_0p312nM - Rep1 with duration: 14.08 seconds.


Creating video for D_0p312nM - Rep1: 100%|██████████| 338/338 [00:24<00:00, 13.71it/s]


Creating video for D_0p156nM - Rep1 with duration: 14.08 seconds.


Creating video for D_0p156nM - Rep1: 100%|██████████| 338/338 [00:23<00:00, 14.21it/s]


In [11]:
conditions = [
    "D_160nM",
    "D_80nM",
    "D_40nM",
    "D_20nM",
    "D_10nM",
    "D_5nM",
    "D_2p5nM",
    "D_1p25nM",
    "D_0p625nM",
    "D_0p312nM",
    "D_0p156nM",
    "negative",
]

conditions

['D_160nM',
 'D_80nM',
 'D_40nM',
 'D_20nM',
 'D_10nM',
 'D_5nM',
 'D_2p5nM',
 'D_1p25nM',
 'D_0p625nM',
 'D_0p312nM',
 'D_0p156nM',
 'negative']

In [12]:
qd.create_combined_heatmap_movie_custom_grid(
    data_path, 
    conditions, 
    subconditions, 
    channel='cy5', 
    grid_rows=3, 
    grid_cols=4, 
    frame_rate=24,
    batch_size=50
    )


Creating video with duration: 14.08 seconds.


Creating combined frames: 100%|██████████| 338/338 [02:45<00:00,  2.04it/s]


Combined video saved to ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/output_data/movies/combined_heatmap_movie_cy5_24fps_338frames.avi


In [13]:
# Call the function
qd.fluorescence_heatmap(
    data_path, 
    conditions, 
    subconditions, 
    channel='GFP', 
    time_interval_list=time_interval_list, 
    vmax=500, 
    skip_frames=8, 
    calibration_curve_paths=calibration_curve_paths, 
    show_scalebar=True,
    )

Processing negative - Rep1: 100%|██████████| 338/338 [00:35<00:00,  9.41it/s]


In [14]:
# Example usage
qd.create_movies(
    data_path, 
    conditions, 
    subconditions, 
    channel='GFP', 
    frame_rate=24,
    skip_frames=1
    )


Creating video for D_160nM - Rep1 with duration: 14.08 seconds.


Creating video for D_160nM - Rep1: 100%|██████████| 338/338 [00:20<00:00, 16.74it/s]


Creating video for D_80nM - Rep1 with duration: 14.08 seconds.


Creating video for D_80nM - Rep1: 100%|██████████| 338/338 [00:20<00:00, 16.77it/s]


Creating video for D_40nM - Rep1 with duration: 14.08 seconds.


...

Creating video for D_0p312nM - Rep1: 100%|██████████| 338/338 [00:19<00:00, 17.15it/s]


Creating video for D_0p156nM - Rep1 with duration: 14.08 seconds.


Creating video for D_0p156nM - Rep1: 100%|██████████| 338/338 [00:19<00:00, 17.29it/s]


Creating video for negative - Rep1 with duration: 14.08 seconds.


Creating video for negative - Rep1: 100%|██████████| 338/338 [00:19<00:00, 17.58it/s]


In [19]:
qd.create_combined_heatmap_movie_custom_grid(
    data_path, 
    conditions, 
    subconditions, 
    channel='GFP', 
    grid_rows=3, 
    grid_cols=4, 
    frame_rate=24,
    batch_size=50
    )


Creating video with duration: 14.08 seconds.


Creating combined frames: 100%|██████████| 338/338 [02:29<00:00,  2.26it/s]


Combined video saved to ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/output_data/movies/combined_heatmap_movie_GFP_24fps_338frames.avi


In [ ]:
qd.delete_temporary_image_directories(data_path, conditions, subconditions)

### Fluorescence Quantification

In [17]:
# Example usage
mw_kda_list = [44.5] * len(conditions)
droplet_volume_list = [2] * len(conditions)
protein_lengths_list = [397] * len(conditions)

qd.quantify_tiffiles(
    data_path, 
    conditions, 
    subconditions, 
    calibration_curve_paths, 
    mw_kda_list, 
    droplet_volume_list, 
    time_interval_list, 
    protein_lengths_list,
    ribosome_count=10**9,
    skip_frames=1,
    subtract_negative=True
)

Calculating sample intensities: 100%|██████████| 2700/2700 [00:04<00:00, 558.11it/s]


('../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/output_data/combined_expression.csv',
 '../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/output_data/mean_expression.csv')

### PIV pre-processing

In [ ]:
conditions

In [21]:
qd.split_tiffs(data_path, conditions, subconditions, channel='cy5', file_interval=2)


Copied every 2th f'cy5' file from ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/D_160nM/Rep1/original into ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/D_160nM/Rep1/cy5-2x.
Copied every 2th f'cy5' file from ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/D_80nM/Rep1/original into ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/D_80nM/Rep1/cy5-2x.
Copied every 2th f'cy5' file from ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/D_40nM/Rep1/original into ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/D_40nM/Rep1/cy5-2x.
Copied every 2th f'cy5' file from ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/D_20nM/Rep

### PIV

In [27]:
# Define feature limits and other parameters
v = 2E-7
velocity_limits = (0, v)
other_limits = (-0.0005, 0.0005)
skip_frames = 1 ### CHANGE THIS TO SKIP FRAMES


velocity_limits = (None, None)
other_limits = (None, None)


feature_limits = {
    # 'u [m/s]': (-v, v), 
    # 'v [m/s]': (-v, v), 
    # 'data type [-]': (None, None),
    'velocity magnitude [m/s]': velocity_limits,
    'vorticity [1/s]': other_limits,
    'divergence [1/s]': other_limits,
    # 'dcev [1]': (0, 250),
    'shear [1/s]': other_limits,
    'strain [1/s]': other_limits,
    'vector direction [degrees]': (-180, 180),
}


# Features for PCA and plotting
features_pca = [
    "vorticity [1/s]_mean",
    "velocity magnitude [um/s]",
    "distance [m]_mean",
    "divergence [1/s]_mean",
    "shear [1/s]_mean",
    "strain [1/s]_mean",
    "correlation length [um]", 
    "power [W]_mean",
    "work [J]",
]


In [33]:
conditions =[
    'D_160nM',
    'D_80nM',
    'D_40nM',
    'D_20nM',
    'D_10nM',
    'D_5nM',
    'D_2p5nM',
    'D_1p25nM',
    'D_0p625nM',
    # 'D_0p312nM',
    # 'D_0p156nM'
 ]

In [34]:
time_interval_list = [8] * len(conditions)  # time intervals in seconds between frames for each condition


# Process PIV data
qd.process_piv_data(
    data_path, 
    conditions,
    subconditions, 
    feature_limits, 
    time_interval_list, 
    min_frame=0, 
    max_frame=None, 
    skip_frames=1, 
    plot_autocorrelation=False, 
    frame_rate=1, 
    heatmaps=False
    )


Processing PIV data:   0%|          | 0/9 [00:00<?, ?it/s]




































































































Processing PIV data:  11%|█         | 1/9 [00:25<03:20, 25.01s/it]


















































































































































































































Processing PIV data:  22%|██▏       | 2/9 [01:21<05:06, 43.82s/it]

























































































































































































Processing PIV data:  33%|███▎      | 3/9 [02:17<04:54, 49.09s/it]

































































































































































































Processing PIV data:  44%|████▍     | 4/9 [03:11<04:

In [ ]:
conditions[:-1]


In [ ]:
# Plot features and PCA
qd.plot_PIV_all(
    data_path, 
    conditions[:-1],
    subconditions, 
    features_pca, 
    min_frame=0, 
    max_frame=None
    )


### Expression + PIV

In [35]:
# Combine the data and save it to the "output_data" directory
qd.combine_averaged_dataframes(data_path, conditions, subconditions)


Conditions: 100%|██████████| 9/9 [00:00<00:00, 45.24it/s]


Combined DataFrame saved to: ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/output_data/combined_PIV.csv


In [36]:
qd.merge_expression_piv_data(data_path)


Merged DataFrame saved to: ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/110324-D_titration-RT/2p5TMB-1ulDNA_1/output_data/merged_expression_PIV.csv


In [ ]:
# x_column = "time (h)"  # Example x-axis column
# y_column = "Protein Concentration_nM"  # Example y-axis column


# x_column = "Protein Concentration_nM"  # Example y-axis column
# y_column = "velocity magnitude [m/s]_mean"  # Example y-axis column

# x_column = "time (h)"  # Example x-axis column
# y_column = "velocity magnitude [m/s]_mean"  # Example y-axis column

# x_column = "time (h)"  # Example x-axis column
# y_column = "power [W]_mean"  # Example y-axis column

# x_column = "time (h)"  # Example x-axis column
# y_column = "work [J]_mean"  # Example y-axis column

x_column = "time (h)"  # Example x-axis column
y_column = "distance [m]_mean"  # Example y-axis column


qd.plot_expression_piv(
    data_path,
    conditions,
    x_column, 
    y_column, 
    sigma_x=0.1, 
    sigma_y=10, 
    x_log=False, 
    y_log=True, 
    min_frame=0, 
    max_frame=None, 
    individual_plots=False
    )

In [ ]:

# List of features for PCA
features_pca = [
    "vorticity [1/s]_mean",
    "velocity magnitude [m/s]_mean",
    "distance [m]_mean",
    "divergence [1/s]_mean",
    "shear [1/s]_mean",
    "strain [1/s]_mean",
    "correlation length [m]_mean", 
    "power [W]_mean",
    "work [J]_mean",
    'vector direction [degrees]_mean',
    "Protein Concentration_nM", 
    'time (min)'
]

# Run PCA and save plot (with all conditions and subconditions in the same plot)
qd.plot_pca_expression_piv(data_path, conditions=conditions, subconditions=subconditions, features=features_pca, sigma=1)


In [ ]:

# Example usage
qd.delete_outputs(data_path, conditions, subconditions, output_dirs=None)
